In [1]:

# Importing the libraries

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torch
from torch.autograd import Variable


In [37]:

# Importing the dataset movies

movies=pd.read_csv('movies.dat',sep='::', header=None, engine = 'python', encoding='latin-1')

movies.head()

In [39]:
# Importing the dataset users

users=pd.read_csv('users.dat',sep='::', header=None, engine = 'python', encoding='latin-1')

users.head()

In [41]:

# Importing the dataset ratings

ratings=pd.read_csv('ratings.dat',sep='::', header=None, engine = 'python', encoding='latin-1')

ratings.head()

In [42]:
#prepare train and test data 

train=pd.read_csv('u11.base', delimiter='\t')

train=np.array(train,dtype='int')

train.dtype

test=pd.read_csv('u11.test', delimiter='\t')

test=np.array(test,dtype='int')

In [47]:
#find the number of user and movies

nb_users=int(max(max(train[:,0]),max(test[:,0])))

nb_movies=int(max(max(train[:,1]),max(test[:,1])))

In [49]:
#preprocessing the input into an array with userids and movies

def convert(data):
    new_data=[]
    for id_users in range(1,nb_users+1):
        id_movies = data[:,1][data[:,0]==id_users]
        id_ratings = data[:,2][data[:,0]==id_users]
        ratings=np.zeros(nb_movies)
        ratings[id_movies-1]=id_ratings
        new_data.append(list(ratings))
    return new_data

        

train=convert(train)

test=convert(test)

In [52]:
#transform the arrays into torch tensors

train=torch.FloatTensor(train)

test=torch.FloatTensor(test)

In [54]:
#encoding the ratings to liked and hated 

train[train==0]=-1
train[train==1]=0
train[train==2]=0
train[train>3]=1

test[test==0]=-1
test[test==1]=0
test[test==2]=0
test[test>=3]=1

In [56]:
 # Creating the RBM Network
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [57]:
#fit the rbm network on the data 

nv = len(train[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

In [61]:

# train the RBM model
nb_epoch = 100
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = train[id_user:id_user+batch_size]
        v0 = train[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: 0.738309168467
epoch: 2 loss: 0.73672665848
epoch: 3 loss: 0.735321356125
epoch: 4 loss: 0.734854222959
epoch: 5 loss: 0.734601580259
epoch: 6 loss: 0.734905409309
epoch: 7 loss: 0.734382533621
epoch: 8 loss: 0.735892582015
epoch: 9 loss: 0.736662881672
epoch: 10 loss: 0.737249751079
epoch: 11 loss: 0.737007958016
epoch: 12 loss: 0.73786823908
epoch: 13 loss: 0.738464362957
epoch: 14 loss: 0.738335813775
epoch: 15 loss: 0.738593420767
epoch: 16 loss: 0.739138265681
epoch: 17 loss: 0.739736768351
epoch: 18 loss: 0.740315328112
epoch: 19 loss: 0.73990730635
epoch: 20 loss: 0.740558909134
epoch: 21 loss: 0.74082710649
epoch: 22 loss: 0.740501617797
epoch: 23 loss: 0.740102625841
epoch: 24 loss: 0.739416807574
epoch: 25 loss: 0.739348358624
epoch: 26 loss: 0.739536368047
epoch: 27 loss: 0.739961204425
epoch: 28 loss: 0.739664800041
epoch: 29 loss: 0.73988725723
epoch: 30 loss: 0.73937598976
epoch: 31 loss: 0.739649474848
epoch: 32 loss: 0.739221191329
epoch: 33 loss: 0.73891

In [62]:
# Test the RBM model
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = train[id_user:id_user+1]
    vt = test[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 0.19290748262
